<a href="https://colab.research.google.com/github/sanjeevrs2000/CS6910-Assignment-2/blob/main/Assignment_2B_Wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install wandb

     |████████████████████████████████| 2.1MB 19.2MB/s 
     |████████████████████████████████| 133kB 59.2MB/s 
     |████████████████████████████████| 102kB 14.0MB/s 
     |████████████████████████████████| 163kB 54.3MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=72fa4abed9d247d7dbf829abd0475440475010eb83387c8f91acad8689402331
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=453a118810ee0e016e0629461da31ff76931a2e5fe57116fd91f725bcf1e10c3
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [1]:
import numpy as np
import pandas as pd
import os
import keras
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2
#import wandb
#from wandb.keras import WandbCallback

In [2]:
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception

In [3]:
%%capture
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip
!unzip nature_12K.zip

train_dir='inaturalist_12K/train/'
test_dir='inaturalist_12K/val/'
categories=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

In [4]:
#Run this cell

sweep_config = {
  'name': 'sweep5b',  
  'method': 'grid',
  'parameters': {
        
        
        'fc_size':{
            'values':[128]
        },
        'batchnorm':{
            'values':['yes']
        },
        'augmentation':{
            'values':['yes']   
        },
        'droprate':{
            'values':[0.4]
        },
        'pre_train':{
            'values':['inceptionresnetv2','Xception']
        }

           
    }

}

#this config gives best val accuracy
config_defaults={
    'fc_size':128,
    'batchnorm':'yes',
    'augmentation':'yes',
    'droprate':0.4,
    'pre_train':'inceptionresnetv2'
}

In [ ]:
#Run this cell for Wandb after uncommenting

def PreTrain():

  #wandb init and log commented to do trial run

  #wandb.init(config=config_defaults)
  #configs=wandb.config

  configs=config_defaults

  batchnorm=configs['batchnorm']
  droprate=configs['droprate']
  augmentation=configs['augmentation']
  fc_size=configs['fc_size']
  pre_train = configs['pre_train']

  #wandb.run.name='model_'+pre_train+'_fc_2_size_'+str(fc_size)+'_droprate_'+str(droprate)+ str('_bn_' if batchnorm=='yes' else '') +str('augment' if augmentation=='yes' else '')


  if pre_train == 'inceptionv3':
    img_height = 299
    img_width = 299
    base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'inceptionresnetv2':
    img_height = 299
    img_width = 299
    base_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'resnet50':
    img_height = 224
    img_width = 224
    base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'Xception':
    img_height = 299
    img_width = 299
    base_model = Xception(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))  


  for layers in base_model.layers:
    layers.trainable = False

  model = keras.Sequential([
      tf.keras.Input(shape=(img_height, img_width,3,)),
      base_model,
      Flatten(),
      Dense(fc_size,activation='relu'),
      
  ])
  if batchnorm == 'yes':
    model.add(BatchNormalization())
  model.add(Dropout(droprate))
  model.add(Dense(fc_size, activation='relu'))
  model.add(Dropout(droprate))
  model.add(Dense(10 ,activation='softmax'))
  

  #data generators for train(with and without validation) and validation
  augment= ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
  )

  datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
  )

  train_set = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    color_mode='rgb',
    class_mode='sparse',
    shuffle=True,
    subset='training',
    seed=123,
  )

  aug_set = augment.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      batch_size=32,
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='training',
      seed=123,
  )

  val_set = datagen.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='validation',
      seed=123,
  )

  model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=[tf.keras.losses.SparseCategoricalCrossentropy()],
    metrics=['accuracy'],
  )

  if augmentation =='no' :
    #hist=model.fit(train_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()]) 
    hist=model.fit(train_set,epochs=10,validation_data=val_set)
  
  else:
    #hist=model.fit(aug_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()])
    hist=model.fit(aug_set,epochs=10,validation_data=val_set)


  val_acc=max(hist.history['val_accuracy'])
  params={'batch_norm':batchnorm,'augmentation':augmentation,'dropout':droprate,'pre_trained_model':pre_train,'val_acc':val_acc}
  #wandb.log(params)
  

In [ ]:
#Ignore this and proceed
#sweep_id=wandb.sweep(sweep_config,entity='sanjeev-nimeesh',project='assignment_2')
#wandb.agent(sweep_id, PreTrain)   

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: xqg252wy
Sweep URL: https://wandb.ai/sanjeev-nimeesh/assignment_2/sweeps/xqg252wy


wandb: Agent Starting Run: p5k0yj6c with config:
wandb: 	augmentation: yes
wandb: 	batchnorm: yes
wandb: 	droprate: 0.4
wandb: 	fc_size: 128
wandb: 	pre_train: inceptionresnetv2
wandb: Currently logged in as: sanjeev-nimeesh (use `wandb login --relogin` to force relogin)


219062272/219055592 [==============================] - 4s 0us/step
Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 455s 1s/step - loss: 1.7072 - accuracy: 0.4624 - val_loss: 0.6156 - val_accuracy: 0.8108
Epoch 2/10
282/282 [==============================] - 412s 1s/step - loss: 0.9484 - accuracy: 0.7123 - val_loss: 0.5922 - val_accuracy: 0.8128
Epoch 3/10
282/282 [==============================] - 397s 1s/step - loss: 0.7925 - accuracy: 0.7502 - val_loss: 0.5909 - val_accuracy: 0.8188
Epoch 4/10
282/282 [==============================] - 399s 1s/step - loss: 0.7881 - accuracy: 0.7508 - val_loss: 0.5601 - val_accuracy: 0.8268
Epoch 5/10
282/282 [==============================] - 399s 1s/step - loss: 0.7495 - accuracy: 0.7676 - val_loss: 0.5386 - val_accuracy: 0.8288
Epoch 6/10
282/282 [==============================] - 397s 1s/step - loss: 0.7232 - accura

...

wandb: Agent Starting Run: vbi92vd8 with config:
wandb: 	augmentation: yes
wandb: 	batchnorm: yes
wandb: 	droprate: 0.4
wandb: 	fc_size: 128
wandb: 	pre_train: Xception


83689472/83683744 [==============================] - 1s 0us/step
Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 383s 1s/step - loss: 1.7602 - accuracy: 0.4521 - val_loss: 0.6814 - val_accuracy: 0.7818
Epoch 2/10
282/282 [==============================] - 374s 1s/step - loss: 0.9588 - accuracy: 0.6956 - val_loss: 0.6375 - val_accuracy: 0.7908
Epoch 3/10
282/282 [==============================] - 370s 1s/step - loss: 0.8332 - accuracy: 0.7468 - val_loss: 0.5940 - val_accuracy: 0.8148
Epoch 4/10
282/282 [==============================] - 372s 1s/step - loss: 0.7453 - accuracy: 0.7629 - val_loss: 0.5789 - val_accuracy: 0.8188
Epoch 5/10
282/282 [==============================] - 363s 1s/step - loss: 0.7218 - accuracy: 0.7686 - val_loss: 0.5661 - val_accuracy: 0.8078
Epoch 6/10
282/282 [==============================] - 367s 1s/step - loss: 0.7133 - accuracy

epoch,9
loss,0.64408
accuracy,0.79011
val_loss,0.56523
val_accuracy,0.81982
_runtime,3721
_timestamp,1618157290
_step,10
best_val_loss,0.54274
best_epoch,7
batch_norm,yes


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▂▂▂▁▁▁▁
accuracy,▁▅▆▇▇▇████
val_loss,█▆▄▃▂▂▃▁▃▂
val_accuracy,▁▃▇█▆▆▇███
_runtime,▁▂▃▃▄▅▆▆▇██
_timestamp,▁▂▃▃▄▅▆▆▇██
_step,▁▂▂▃▄▅▅▆▇▇█
dropout,▁
val_acc,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
#Trial run with config defaults
PreTrain()

Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 340s 1s/step - loss: 1.7277 - accuracy: 0.4585 - val_loss: 0.6189 - val_accuracy: 0.8128
Epoch 2/10
282/282 [==============================] - 289s 1s/step - loss: 0.9558 - accuracy: 0.7016 - val_loss: 0.5813 - val_accuracy: 0.8268
Epoch 3/10
282/282 [==============================] - 283s 1s/step - loss: 0.8227 - accuracy: 0.7421 - val_loss: 0.5640 - val_accuracy: 0.8338
Epoch 4/10
282/282 [==============================] - 284s 1s/step - loss: 0.7719 - accuracy: 0.7543 - val_loss: 0.5426 - val_accuracy: 0.8388
Epoch 5/10
282/282 [==============================] - 285s 1s/step - loss: 0.7155 - accuracy: 0.7682 - val_loss: 0.5163 - val_accuracy: 0.8458
Epoch 6/10
282/282 [==============================] - 285s 1s/step - loss: 0.6831 - accuracy: 0.7802 - val_loss: 0.5186 - val_accuracy: 0.8308
Epoch 7/10
282

In [5]:
# Defining model with best configs to evaluate on test set  
  configs=config_defaults

  batchnorm=configs['batchnorm']
  droprate=configs['droprate']
  augmentation=configs['augmentation']
  fc_size=configs['fc_size']
  pre_train = configs['pre_train']

  #wandb.run.name='model_'+pre_train+'_fc_2_size_'+str(fc_size)+'_droprate_'+str(droprate)+ str('_bn_' if batchnorm=='yes' else '') +str('augment' if augmentation=='yes' else '')


  if pre_train == 'inceptionv3':
    img_height = 299
    img_width = 299
    base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'inceptionresnetv2':
    img_height = 299
    img_width = 299
    base_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'resnet50':
    img_height = 224
    img_width = 224
    base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'Xception':
    img_height = 299
    img_width = 299
    base_model = Xception(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))  


  for layers in base_model.layers:
    layers.trainable = False

  model = keras.Sequential([
      tf.keras.Input(shape=(img_height, img_width,3,)),
      base_model,
      Flatten(),
      Dense(fc_size,activation='relu'),
      
  ])
  if batchnorm == 'yes':
    model.add(BatchNormalization())
  model.add(Dropout(droprate))
  model.add(Dense(fc_size, activation='relu'))
  model.add(Dropout(droprate))
  model.add(Dense(10 ,activation='softmax'))
  

  #data generators for train(with and without validation) and validation
  augment= ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
  )

  datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
  )

  train_set = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    color_mode='rgb',
    class_mode='sparse',
    shuffle=True,
    subset='training',
    seed=123,
  )

  aug_set = augment.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      batch_size=32,
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='training',
      seed=123,
  )

  val_set = datagen.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='validation',
      seed=123,
  )

  model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=[tf.keras.losses.SparseCategoricalCrossentropy()],
    metrics=['accuracy'],
  )

  if augmentation =='no' :
    #hist=model.fit(train_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()]) 
    hist=model.fit(train_set,epochs=10,validation_data=val_set)
  
  else:
    #hist=model.fit(aug_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()])
    hist=model.fit(aug_set,epochs=10,validation_data=val_set)


219062272/219055592 [==============================] - 1s 0us/step
Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 344s 1s/step - loss: 1.7028 - accuracy: 0.4703 - val_loss: 0.6269 - val_accuracy: 0.8028
Epoch 2/10
282/282 [==============================] - 290s 1s/step - loss: 0.9174 - accuracy: 0.7152 - val_loss: 0.5931 - val_accuracy: 0.8188
Epoch 3/10
282/282 [==============================] - 283s 1s/step - loss: 0.8025 - accuracy: 0.7481 - val_loss: 0.5447 - val_accuracy: 0.8368
Epoch 4/10
282/282 [==============================] - 282s 1s/step - loss: 0.7885 - accuracy: 0.7509 - val_loss: 0.5423 - val_accuracy: 0.8248
Epoch 5/10
282/282 [==============================] - 283s 1s/step - loss: 0.7350 - accuracy: 0.7631 - val_loss: 0.5582 - val_accuracy: 0.8328
Epoch 6/10
282/282 [==============================] - 283s 1s/step - loss: 0.7117 - accura

In [6]:
#data generator for test set
test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
  )

test_set = test_datagen.flow_from_directory(
      test_dir,
      target_size=(img_height, img_width),
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      seed=123,
  )

#evaluating model on test set
model.evaluate(test_set)

Found 2000 images belonging to 10 classes.
63/63 [==============================] - 34s 534ms/step - loss: 0.5280 - accuracy: 0.8410


[0.5280486941337585, 0.8410000205039978]